# **Installing Required Packages**

In [ ]:
!pip install langchain-google-genai google-generativeai langchain langchain-community

In [ ]:
!pip install faiss-cpu

In [ ]:
!pip install google-search-results

In [ ]:
!pip install langchain_classic langchain-community

# **Loading API keys from Colab Secrets**

In [ ]:
from google.colab import userdata
gkey = userdata.get('GAPI')
skey = userdata.get('SERP_KEY')

# **Simple Agent that call tools based on the input**
Here the tools are mocked with functions


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.tools import Tool
from langchain_classic.agents import initialize_agent, AgentType, load_tools, AgentExecutor

def calc_tool(expr: str) -> str:
        return str(eval(expr))

def search_tool(q: str) -> str:
    return "SEARCH_RESULT: product X released in 2010"

t1=Tool(name="search", func=search_tool, description="Search for product info and return results.")
t2=Tool(name="calc", func=calc_tool, description="Evaluate arithmetic expressions safely.")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",temperature=0,
    google_api_key=gkey)

agent = initialize_agent(
    tools=[t1,t2],llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

print(agent.run('Find the year product X released (search), then compute years since release (calc).'))


# **Simple Agent that call tools based on the input**
Here the tools are actual api endpoinds serpapi and llm-math


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, load_tools

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",temperature=0,
    google_api_key=gkey
)
tools = load_tools(
    ["serpapi", "llm-math"],llm=llm,
    serpapi_api_key=skey
)
agent = initialize_agent(
    tools=tools,llm=llm,agent="zero-shot-react-description",verbose=True
)

# Run the query
result = agent.run("What's the weather in Paris and square root of 123?")
print(result)


## **Agentic RAG : An Agent Based RAG Application**


# LangChain imports

In [ ]:
import os
import ast
import operator as op
from typing import List
from pathlib import Path


from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_classic.tools import Tool
from langchain_classic.agents import initialize_agent, AgentType, load_tools
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_classic.chains import RetrievalQA
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic import LLMChain, PromptTemplate



# Document loading & FAISS index


In [ ]:
def calc_tool(expr: str) -> str:
  res = eval(expr)
  return str(res)


def load_documents(doc_folder: str) -> List[str]:
    """
    Simple loader: reads .txt files from directory and returns list[str].
    Replace/extend this to read PDFs, DOCX, or use langchain.document_loaders.
    """
    texts = []
    p = Path(doc_folder)
    for f in p.glob("**/*.txt"):
        texts.append(f.read_text(encoding="utf-8"))
    return texts

def build_faiss_from_texts(texts: List[str], persist_dir: str = "faiss_index"):
    """
    - Splits large texts to chunks
    - Builds embeddings using HuggingFaceEmbeddings (all-MiniLM)
    - Creates FAISS index (persist optional)
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
    docs = []
    for t in texts:
        docs.extend(splitter.split_text(t))

    embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    faiss_db = FAISS.from_texts(docs, embed)

    # optional persist
    try:
        faiss_db.save_local(persist_dir)
    except Exception:
        pass
    return faiss_db

# -----------------------
# Retriever tool wrapper
# -----------------------
def make_retrieval_tool(faiss_db, llm):
    """
    Wrap retrieval + LLM as a tool: user gives a query, tool returns grounded summary
    """
    retriever = faiss_db.as_retriever(search_kwargs={"k": 4})
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

    def retrieval_tool(query: str) -> str:
        try:
            return qa_chain.run(query)
        except Exception as e:
            return f"ERROR during retrieval: {e}"

    return retrieval_tool

# Mock / real search tool (here we use SerpAPI wrapper via load_tools)

In [ ]:
def make_search_tool(llm, serpapi_key: str):
    """
    Uses LangChain's built-in SerpAPI tool via load_tools for web search.
    If serpapi key not present, returns a simple mock function.
    """
    if serpapi_key:
        tools = load_tools(["serpapi"], llm=llm, serpapi_api_key=serpapi_key)
        # load_tools returns a list; find the serpapi tool
        for t in tools:
            if getattr(t, "name", "").lower().startswith("serpapi"):
                return t.func
    # fallback mock
    def mock_search(q: str) -> str:
        return f"MOCK_SEARCH_RESULT for: {q}"
    return mock_search


# Main assembly of agent


In [ ]:

def build_agent(
    doc_folder: str,
    google_api_key: str,
    serpapi_key: str = None
):
    # 1) LLM (Gemini)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0,
        google_api_key=google_api_key
    )

    # 2) Build or load FAISS from local docs
    texts = load_documents(doc_folder)
    if not texts:
        print("Warning: no local docs found. FAISS will be empty (use small demo text).")
        texts = ["Product X launched in 2010. Sales grew from 1.2M to 2.0M in 2015-2020."]
    faiss_db = build_faiss_from_texts(texts)

    # 3) Create retrieval tool
    retrieval_tool_fn = make_retrieval_tool(faiss_db, llm)
    retrieval_tool = Tool(
        name="doc_retriever",
        func=retrieval_tool_fn,
        description="Use this to retrieve and summarize relevant passages from local documents. Input: a natural language question."
    )

    # 4) Search tool (SerpAPI or mock)
    search_fn = make_search_tool(llm, serpapi_key)
    search_tool = Tool(
        name="web_search",
        func=search_fn,
        description="Search the web and return short factual snippets about the query."
    )

    # 5) Calculator
    calc_tool_obj = Tool(
        name="calculator",
        func=calc_tool,
        description="Safely evaluate arithmetic expressions."
    )

    # 6) Memory (short-term conversation buffer)
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)

    # 7) Initialize agent with tools
    tools = [retrieval_tool, search_tool, calc_tool_obj]

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        memory=memory
    )
    return agent, faiss_db

# Example usage

In [ ]:

if __name__ == "__main__":

    # point to folder with .txt docs (or leave empty to trigger demo text)
    DOC_FOLDER = "/content/data_store"

    agent, db = build_agent(DOC_FOLDER, google_api_key=gkey, serpapi_key=skey)
    # Complex multi-step prompt example:
    prompt = (
    "Task: Prepare a concise analytical summary based on the Sherlock Holmes corpus provided. "
    "1) Use doc_retriever to identify the title and collection in which the story 'A Scandal in Bohemia' appears. "
    "2) Retrieve key character information about Sherlock Holmes and Irene Adler from the documents. "
    "3) Extract one example of Holmes’s reasoning or deduction described in the text. "
    "4) Produce a 4-bullet synthesis summarizing the narrative context, main characters, "
    "Holmes’s analytical style, and the significance of the story within the Sherlock Holmes series."
    )

    print("\n----- RUNNING AGENT -----\n")
    output = agent.invoke(prompt)
    print("\n----- AGENT OUTPUT -----\n")
    print(output)


## **Agentic CHAT BOT : An Agent Based Converational RAG Application**

In [ ]:
import os
import ast
import operator as op
from pathlib import Path
from typing import List

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_classic.agents import initialize_agent, AgentType, load_tools
from langchain_classic.tools import Tool
from langchain_classic.chains import RetrievalQA



def calc_tool_fn(expr: str) -> str:
    try:
        return str(eval(expr))
    except Exception as e:
        return f"CALC_ERROR: {e}"

# ---------------
# Document loading & FAISS
# ---------------
def load_txts(folder: str) -> List[str]:
    p = Path(folder)
    texts = []
    if not p.exists():
        return texts
    for f in p.glob("**/*.txt"):
        try:
            texts.append(f.read_text(encoding="utf-8"))
        except Exception:
            pass
    return texts

def build_faiss(texts: List[str], persist_dir: str = "faiss_agentic"):
    splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120)
    chunks = []
    for t in texts:
        chunks.extend(splitter.split_text(t))
    embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_texts(chunks, embed)
    try:
        db.save_local(persist_dir)
    except Exception:
        pass
    return db

# ---------------
# Build Gemini Agent with RAG + Calculator Tool
# ---------------
def build_agent(faiss_db, llm):
    retriever = faiss_db.as_retriever(search_kwargs={"k": 4})
    qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=retriever)

    tools = [
        Tool(
            name="retrieval",
            func=lambda q: qa_chain.invoke({"query": q}),
            description="Useful for answering questions from local documents."
        ),
        Tool(
            name="calculator",
            func=calc_tool_fn,
            description="Useful for evaluating basic arithmetic expressions like '123 + 456'."
        )
    ]

    agent = initialize_agent(
        tools=tools,
        llm=llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )
    return agent

# ---------------
# Main
# ---------------
def main():

    docs = load_txts("/content/data_store")
    if not docs:
        print("No docs found in ./docs — using default text")
    print("Building FAISS index...")
    db = build_faiss(docs)
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0, google_api_key=gkey)
    agent = build_agent(db, llm)


    while True:
        query = input("You: ")
        if query.lower() in ['exit', 'quit']:
            print("Goodbye!")
            break
        answer = agent.invoke({"input": query})
        print(f"Bot: {answer}\n")


if __name__ == "__main__":
    main()

## **LangGraph**

## A Simple LLM Wrapper

In [ ]:
# minimal_langgraph_gemini.py
import os
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI

# -------- State --------
class State(BaseModel):
    input: str = Field(...)
    output: str | None = None

# -------- Gemini LLM --------
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=gkey
)

# -------- Node --------
def respond(state: State) -> State:
    raw = llm.invoke(state.input)
    # minimal normalization without helpers
    if hasattr(raw, "content"):
        text = raw.content
    else:
        text = str(raw)
    return State(input=state.input, output=text)

# -------- Build Graph --------
graph = StateGraph(State)
graph.add_node("respond", respond)
graph.set_entry_point("respond")
graph.add_edge("respond", END)
app = graph.compile()

# -------- Run --------
result = app.invoke({"input": "Explain LangGraph in one line"})
print(result['output'])


## A Mock Question Answering

In [ ]:
from pydantic import BaseModel, Field
from langgraph.graph import StateGraph, END

class State(BaseModel):
    query: str = Field(...)
    action: str | None = None
    docs: list | None = None
    answer: str | None = None

# mock db & llm as above
class MockDB:
    def similarity_search(self, q):
        return [type("D", (), {"page_content": "Product X released in 2010."})()]

class MockLLM:
    def invoke(self, prompt):
        return f"ANSWER_FROM_LLM: {prompt[:80]}..."

db = MockDB()
llm = MockLLM()

def planner(state: State):
    q = state.query
    state.action = "retrieve" if ("year" in q or "released" in q) else "answer_direct"
    return state

def retrieve(state: State):
    state.docs = db.similarity_search(state.query)
    return state

def answer(state: State):
    context = "\n".join([d.page_content for d in (state.docs or [])])
    prompt = f"Context:\n{context}\n\nQuestion:\n{state.query}"
    raw = llm.invoke(prompt)
    text = getattr(raw, "content", raw)
    state.answer = str(text)
    return state

graph = StateGraph(State)
graph.add_node("planner", planner)
graph.add_node("retriever", retrieve)
graph.add_node("answer", answer)

graph.set_entry_point("planner")
graph.add_conditional_edges("planner",
    lambda s: s.action,
    {"retrieve": "retriever", "answer_direct": "answer"}
)
graph.add_edge("retriever", "answer")
graph.add_edge("answer", END)

app = graph.compile()

# Must pass 'query' when invoking because Pydantic requires it
result = app.invoke({"query": "When was Product X released?"})

print("answer:", result['answer'])


## **Multi Agent Systems**

In [ ]:
class MASState(BaseModel):
    query: str
    docs: list | None = None
    draft: str | None = None
    final: str | None = None
    action: str | None = None

# --- Planner Agent ---
def planner(state):
    if "year" in state.query or "released" in state.query:
        state.action = "retrieve"
    else:
        state.action = "reason_direct"
    return state

# --- Retriever Agent ---
def retriever_agent(state):
    state.docs = db.similarity_search(state.query)
    return state

# --- Reasoner Agent (Gemini) ---
def reasoner_agent(state):
    ctx = "\n".join(d.page_content for d in (state.docs or []))
    prompt = f"Context: {ctx}\nQuestion: {state.query}\nAnswer:"
    raw = llm.invoke(prompt)
    state.draft = str(getattr(raw, "content", raw))
    return state

# --- Critic Agent ---
def critic_agent(state):
    critique = llm.invoke(
        f"Evaluate this answer:\n{state.draft}\nProvide corrections only."
    )
    state.final = str(getattr(critique, "content", critique))
    return state

# --- Build Graph ---
graph = StateGraph(MASState)
graph.add_node("planner", planner)
graph.add_node("retriever", retriever_agent)
graph.add_node("reasoner", reasoner_agent)
graph.add_node("critic", critic_agent)

graph.set_entry_point("planner")
graph.add_conditional_edges("planner",
    lambda s: s.action,
    {"retrieve": "retriever", "reason_direct": "reasoner"}
)
graph.add_edge("retriever", "reasoner")
graph.add_edge("reasoner", "critic")
graph.add_edge("critic", END)

app = graph.compile()

In [ ]:
result = app.invoke({"query": "When was Product X released?"})

print("Draft answer:", result['draft'])
print("Final corrected answer:", result['final'])